# Pickle 模块测试套件 - 稳定性与正确性分析


In [27]:
import pickle
import hashlib
import sys
import datetime
import pickle
import sys
from typing import Any, Dict, List


In [28]:
def get_pickle_hash(obj):
       try:
           data = pickle.dumps(obj)
           return hashlib.sha256(data).hexdigest()
       except Exception as e:
           return f"ERROR: {e}"

## 白盒测试


In [26]:
def create_circular_reference():
    a = []
    a.append(a)  # 创建循环引用
    return a

# 1. 语句覆盖测试
def test_statement_coverage():
    print("=== 语句覆盖测试 ===")
    test_cases = {
        "基本序列化": 42,
        "不可序列化对象": object(),
        "循环引用": create_circular_reference(),
        "文件对象": open("test.txt", "w"),
        "线程对象": threading.Thread(),
        "协议版本": 42
    }
    
    for name, obj in test_cases.items():
        try:
            if name == "协议版本":
                result = pickle.dumps(obj, protocol=4)
            else:
                result = pickle.dumps(obj)
            print(f"测试用例: {name:<15} | 结果: 成功")
        except Exception as e:
            print(f"测试用例: {name:<15} | 结果: 失败 - {str(e)}")


# 2. 条件覆盖测试
def test_condition_coverage():
    print("\n=== 条件覆盖测试 ===")
    test_cases = {
        "简单对象": 42,
        "复杂对象": {"a": [1, 2, 3]},
        "空对象": None,
        "大对象": "x" * 1000
    }
    
    for name, obj in test_cases.items():
        try:
            result = pickle.dumps(obj)
            print(f"测试用例: {name:<15} | 结果: 成功")
        except Exception as e:
            print(f"测试用例: {name:<15} | 结果: 失败 - {str(e)}")

# 3. 路径覆盖测试
def test_path_coverage():
    print("\n=== 路径覆盖测试 ===")
    def create_nested_structure(depth):
        if depth == 0:
            return "leaf"
        return {"nested": create_nested_structure(depth - 1)}
    
    for depth in [1, 10, 100]:
        try:
            obj = create_nested_structure(depth)
            result = pickle.dumps(obj)
            print(f"嵌套深度 {depth:<3} | 结果: 成功")
        except Exception as e:
            print(f"嵌套深度 {depth:<3} | 结果: 失败 - {str(e)}")


# 运行所有测试
def run_all_tests():
    print("开始运行白盒测试套件...\n")
    test_statement_coverage()
    test_condition_coverage()
    test_path_coverage()
    print("\n测试套件运行完成！")

# 执行测试
run_all_tests()

开始运行白盒测试套件...

=== 语句覆盖测试 ===
测试用例: 基本序列化           | 结果: 成功
测试用例: 不可序列化对象         | 结果: 成功
测试用例: 循环引用            | 结果: 成功
测试用例: 文件对象            | 结果: 失败 - cannot pickle '_io.TextIOWrapper' object
测试用例: 线程对象            | 结果: 失败 - cannot pickle '_thread.lock' object
测试用例: 协议版本            | 结果: 成功

=== 条件覆盖测试 ===
测试用例: 简单对象            | 结果: 成功
测试用例: 复杂对象            | 结果: 成功
测试用例: 空对象             | 结果: 成功
测试用例: 大对象             | 结果: 成功

=== 路径覆盖测试 ===
嵌套深度 1   | 结果: 成功
嵌套深度 10  | 结果: 成功
嵌套深度 100 | 结果: 成功

测试套件运行完成！


## 黑盒测试

## 等价类划分

In [1]:
test_cases = {
    "普通整数": 42,
    "浮点数": 3.14159,
    "布尔值True": True,
    "布尔值False": False,
    "列表对象": [1, 2, 3],
    "元组对象": (4, 5, 6),
    "字典对象": {"a": 1, "b": 2},
    "集合对象": {7, 8, 9},
    "特殊字符": "!@#$%^&*()_+{}|:<>?",
    "Unicode字符串": "你好，世界！"
}

print("等价类测试结果：")
print("=" * 80)
for name, obj in test_cases.items():
    try:
        hash_result = get_pickle_hash(obj)
        is_valid = isinstance(hash_result, str) and len(hash_result) == 64
        print(f"测试用例: {name:<15} | 对象: {str(obj)[:30]:<30} | 哈希: {hash_result} | 通过: {is_valid}")
    except Exception as e:
        print(f"测试用例: {name:<15} | 对象: {str(obj)[:30]:<30} | 错误: {str(e)} | 通过: False")
print("=" * 80)

等价类测试结果：
测试用例: 普通整数            | 对象: 42                             | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 浮点数             | 对象: 3.14159                        | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 布尔值True         | 对象: True                           | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 布尔值False        | 对象: False                          | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 列表对象            | 对象: [1, 2, 3]                      | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 元组对象            | 对象: (4, 5, 6)                      | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 字典对象            | 对象: {'a': 1, 'b': 2}               | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 集合对象            | 对象: {8, 9, 7}                      | 错误: name 'get_pickle_hash' is not defined | 通过: False
测试用例: 特殊字符            | 对象: !@#$%^&*()_+{}|:<>?            | 错误: name '

## 边界情况测试

In [18]:
def create_deep_dict(depth):
    if depth == 0:
        return "leaf"
    return {"nested": create_deep_dict(depth - 1)}

def run_boundary_tests():
    test_cases = {
        "空对象": None,
        "大对象": {"data": "x" * 1000000},
        "空字符串": "",
        "最大整数": sys.maxsize,
        "最小浮点数": sys.float_info.min,
        "深度嵌套对象": create_deep_dict(100),
        "重复数据对象": {"data": ["x"] * 10000}
    }

    print("边界值测试结果：")
    print("=" * 80)
    for name, obj in test_cases.items():
        try:
            hash_result = get_pickle_hash(obj)
            is_valid = isinstance(hash_result, str) and len(hash_result) == 64
            print(f"测试用例: {name:<15} | 对象: {str(obj)[:30]:<30} | 哈希: {hash_result} | 通过: {is_valid}")
        except Exception as e:
            print(f"测试用例: {name:<15} | 对象: {str(obj)[:30]:<30} | 错误: {str(e)} | 通过: False")
    print("=" * 80)

run_boundary_tests()        

边界值测试结果：
测试用例: 空对象             | 对象: None                           | 哈希: 9c298d589a2158eb513cb52191144518a2acab2cb0c04f1df14fca0f712fa4a1 | 通过: True
测试用例: 大对象             | 对象: {'data': 'xxxxxxxxxxxxxxxxxxxx | 哈希: 23dc9c3c405badcf8444a571d656d3c5dbbf647285e033c00cf6df967206d6d6 | 通过: True
测试用例: 空字符串            | 对象:                                | 哈希: 8e2a8bd996c29b99c909664ee51eb5ee3b2358306a4269d5b69dd99075e6dd85 | 通过: True
测试用例: 最大整数            | 对象: 9223372036854775807            | 哈希: a6d2c4015b1ac7cdb57d88d220317a5fc2ee30d3e9429988ff53f54b39f1ca97 | 通过: True
测试用例: 最小浮点数           | 对象: 2.2250738585072014e-308        | 哈希: 2f639def69a32cf0165f9ed1981d5ebf1a96b2b191001ce3011bf6f047af962d | 通过: True
测试用例: 深度嵌套对象          | 对象: {'nested': {'nested': {'nested | 哈希: 4d30d91fcb5e2dcf10a85aa8731a850a9a903b70067df4d9732ffb855d082a31 | 通过: True
测试用例: 重复数据对象          | 对象: {'data': ['x', 'x', 'x', 'x',  | 哈希: fa871e2ce0e07a5554e8bf50e43b307320c556426d488ed08153dc6a0ba283ea | 通过: True


## 模糊测试示例：随机生成复杂对象并检查序列化稳定性


In [19]:
import random

def generate_random_nested_structure(depth=3):
    if depth == 0:
        return random.choice([42, 3.14, True, "text", None])
    structure_type = random.choice(["list", "dict", "tuple"])
    if structure_type == "list":
        return [generate_random_nested_structure(depth - 1) for _ in range(2)]
    elif structure_type == "dict":
        return {str(i): generate_random_nested_structure(depth - 1) for i in range(2)}
    else:
        return tuple(generate_random_nested_structure(depth - 1) for _ in range(2))

fuzz_object = generate_random_nested_structure()
hash_before = get_pickle_hash(fuzz_object)
hash_after = get_pickle_hash(fuzz_object)

print("生成的结构:", fuzz_object)
print("第一次序列化哈希:", hash_before)
print("第二次序列化哈希:", hash_after)
print("哈希一致性:", hash_before == hash_after)

生成的结构: [{'0': {'0': 42, '1': True}, '1': {'0': 42, '1': 3.14}}, (('text', 3.14), [3.14, None])]
第一次序列化哈希: 55a738e546d03f3163b5a0a4ea27350be29371e57ca269d5ce3c3519bce3ea3c
第二次序列化哈希: 55a738e546d03f3163b5a0a4ea27350be29371e57ca269d5ce3c3519bce3ea3c
哈希一致性: True


## 哈希一致性测试：不同环境下相同对象的 Pickle 哈希值是否一致


In [29]:
sample_object = {"test": [1, 2, 3], "time": datetime.datetime(2020, 1, 1)}

hash1 = get_pickle_hash(sample_object)
hash2 = get_pickle_hash(sample_object)
hash3 = get_pickle_hash(sample_object)

print("第一次哈希:", hash1)
print("第二次哈希:", hash2)
print("第三次哈希:", hash3)
print("一致性:", hash1 == hash2 == hash3)

第一次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
第二次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
第三次哈希: 415f39ea4600f7fee567c52c63c008d0dda456cc3fe918cb6b0f086918545b6b
一致性: True
